In [16]:
import pyspark
import pandas as pd
import matplotlib.pyplot as plt
import os
from datetime import datetime, timedelta
import numpy as np
from scipy.interpolate import interp1d
import time

In [2]:
def set_pyspark():
    myConf = pyspark.SparkConf()
    spark = pyspark.sql.SparkSession.builder\
        .master("local")\
        .appName("myApp")\
        .config(conf = myConf)\
        .getOrCreate()
    return spark

## 데이터 불러오기 & 전처리

In [3]:
def data_load(file_name, date, label, spark):
    data_df = pd.read_excel(os.path.join("data", file_name), names=["data"])
    data_list = list(data_df["data"])
    
    temp = []
    data_temp = []
    for i in range(len(data_list)):
        temp.append(data_list[i])
        if i % 2 != 0:
            data_temp.append(temp)
            temp = []
    
    dataRDD = spark.sparkContext.parallelize(data_temp)
    
    changeData = dataRDD.map(lambda x: (x[0].split(), x[1]))\
                    .map(lambda x: (date + " " + x[0][2], x[1]))\
                    .map(lambda x: (x[0].replace(" ", " 0"), x[1]) if len(x[0]) == 15 else (x[0], x[1]))\
                    .sortByKey(lambda x: x[0])
    
    time_stamp = changeData.map(lambda x: x[0][-5:]).collect()
    data_count = changeData.map(lambda x: x[1]).collect()
    
    return time_stamp, data_count, changeData

## 선형보간전 데이터 전처리

In [4]:
def before_linear(data, date, spark):
    _data = data.collect()
    _f, _e = _data[0][1], _data[-1][1]
    _data = dict(_data)
    
    start_time = date + " 00:00"
    end_time = date + " 23:59"
    
    _data[start_time] = _f
    _data[end_time] = _e

    _data = sorted(_data.items(), key = lambda x: x[0])
    _data = [k if len(k[0]) != 15 else [] for k in _data]
    _data = list(filter(lambda x: len(x[0]) >= 13, _data))
    return _data

## 선형 보간 함수

In [5]:
def isitRight(step_data):
    step_data = sorted(step_data)
    for i in range(len(step_data) - 1):
        try:
            now = datetime.strptime(step_data[i][0], "%Y-%m-%d %H:%M")
            nxt = datetime.strptime(step_data[i+1][0], "%Y-%m-%d %H:%M")
        except:
            continue
            
        now_value = step_data[i][1]
        nxt_value = step_data[i+1][1]

        new_t = (nxt - now) / 2
        # nn분 30초 나오는 애들은 30초를 버리고
        # nn분만 살려서 가는 형태로 갑시다
        if new_t < timedelta(minutes = 1):
            continue
        else:
            if new_t % timedelta(seconds = 60) != timedelta(seconds = 0):
                new_t -= timedelta(seconds = 30)

            new_value = (now_value + nxt_value) / 2
            new_t = now + new_t
            
            new_t.strftime("%Y-%m-%d %H:%M")
            new_t = str(new_t)
            new_t = new_t[:-3]
            step_data.append((new_t, new_value))
            
    return step_data

In [73]:
import numpy as np

In [87]:
def yesItIsRight(data):
    startPoint = time.time()     
    while len(data) < 1440:
        data = isitRight(data)
        finishPoint = time.time() 
        if finishPoint - startPoint >= 5:
            print(len(data))
            data = list(map(lambda x: (int(x[0][-5:-3]) * 60) + int(x[0][-2:]), data))
            for d in data:
                if d not in [i for i in range(1440)]:
                    print(d)
#             data = list(filter(lambda x: x not in [i for i in range(1440)], data))
#             print(data)
#             ddd = np.array([i for i in range(1440)])
#             ccc = np.array(data)
            print(len(data))
            print(sorted(set([i for i in range(1440)]) - set(data)))
            print(len(set(data)))
            break
    data = sorted(data, key = lambda x: x[0])
    return data

## 그래프 그리기 함수

In [7]:
def draw_graph(data, label, points):
    _x = spark.sparkContext.parallelize(data).map(lambda x: x[0]).collect()
    _y = spark.sparkContext.parallelize(data).map(lambda x: x[1]).collect()
    
    plt.plot(_x[:], _y[:], label = label)
    plt.rcParams['figure.figsize'] = (20, 4)
    plt.rcParams['font.size'] = 10
    plt.legend()

## 전체 기능 함수

In [26]:
def load_and_linear(date, file_name, label, spark):
    _t, _d, _a = data_load(file_name, date, label, spark)
    _dlinear = before_linear(_a, date, spark)
    print()
    print("#"*100)
    _all = yesItIsRight(_dlinear)
    print(len(_all))
    return _all

## 데이터 저장

In [9]:
def save_data(data, file_name):
    all_df = pd.DataFrame(data, columns= ["hearts", "steps"])
    all_df.to_csv(os.path.join("data", file_name))
    return all_df

In [10]:
def merge_data(h_d, s_d):
    return [[v[1], s_d[i][1]] for i, v in enumerate(h_d)]

In [11]:
def final(date, _date, spark):
    h, s = "hearts", "steps"
    hearts_file_name = _date + "_" + h + ".xlsx"
    steps_file_name = _date + "_" + s + ".xlsx"
    
    heart_data = load_and_linear(date, hearts_file_name, h, spark)
    step_data = load_and_linear(date, steps_file_name, s, spark)
    
    merged = merge_data(heart_data, step_data)
    save_file_name = "all_day_" + _date + ".csv"
    ans = save_data(merged, save_file_name)
    return ans

## 실행파트

In [12]:
spark = set_pyspark()

22/01/20 14:26:45 WARN Utils: Your hostname, jeongxoo.local resolves to a loopback address: 127.0.0.1; using 172.20.10.4 instead (on interface en0)
22/01/20 14:26:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/01/20 14:26:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [101]:
date = input()
_date = date[2:]
date = date[:4] + "-" + date[4:6] + "-" + date[-2:]

20211118


In [102]:
final(date, _date, spark)

/usr/local/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling



####################################################################################################
1440

####################################################################################################
1440


,hearts,steps
0,72.0,113.0
1,72.0,113.0
2,72.0,113.0
3,72.0,113.0
4,72.0,113.0
...,...,...
1435,86.0,40.0
1436,86.0,40.0
1437,86.0,40.0
1438,86.0,40.0
